In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


구글 드라이브를 연결하여 데이터를 저장하고 읽을 수 있게 합니다.

In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from tqdm import tqdm

필요한 라이브러리들을 불러옵니다.
torch는 PyTorch, pandas, numpy는 데이터 처리, PIL은 이미지 처리, tqdm은 진행 상황을 나타내는 라이브러리입니다.

In [2]:
class SemiconductorDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.file_names = sorted(os.listdir(folder_path))
        self.transform = transform

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.folder_path, self.file_names[idx])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, 0  # label은 필요하지 않음

데이터셋을 정의하는 클래스입니다.
이미지가 저장된 폴더에서 파일 이름을 읽고, 이미지를 로드하여 전처리 후 반환합니다.
label은 Anomaly Detection에서는 필요하지 않지만, 코드에서는 0을 반환하도록 설정되어 있습니다.

Anomaly Detection에서 label은 보통 모델의 학습에 필요하지 않습니다. 대신, 모델은 정상 이미지를 학습하고, 이후에 **재구성 오차(혹은 anomaly score)**를 통해 이상 여부를 판별합니다.
데이터셋의 각 샘플에 대해 레이블을 필요로 하지 않으므로, __getitem__ 메서드에서 label을 반환할 필요가 없습니다. 이 코드는 레이블을 0으로 설정하여, 레이블 정보가 없다는 것을 명시적으로 나타냅니다.

Anomaly Detection에서는 레이블이 필요하지 않기 때문에, 해당 코드에서는 레이블을 사용하지 않음을 나타내기 위해 0을 반환합니다.
이후 anomaly score를 통해 이상 이미지인지 정상 이미지인지 판별하게 됩니다.

In [ ]:
# 데이터 경로 설정
train_folder = "/content/drive/MyDrive/open/train"
test_folder = "/content/drive/MyDrive/open/test"

# 이미지 전처리 파이프라인
image_size = 256
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # [-1, 1] 정규화
])

# Dataset 및 DataLoader 생성
train_dataset = SemiconductorDataset(train_folder, transform=transform)
test_dataset = SemiconductorDataset(test_folder, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

이미지 크기를 256x256으로 변경하고, 텐서로 변환한 후, [-1, 1] 범위로 정규화합니다.
학습 및 테스트 데이터셋을 DataLoader에 넣어 배치 단위로 처리할 수 있도록 합니다.

텐서로 변환한다"는 것은 데이터를 텐서(Tensor) 형태로 변환하는 것을 의미합니다. 텐서는 다차원 배열(즉, 행렬)을 일반화한 개념으로, 숫자나 데이터를 처리하기 위한 구조입니다. 텐서는 데이터의 형태와 차원을 다룰 수 있게 해 주며, 딥러닝 모델에서 데이터를 입력하거나 출력할 때 사용됩니다.

고차원 텐서(3D 이상): 여러 개의 행렬들이 모인 데이터 구조 (예: 이미지 데이터는 보통 3D 텐서로 표현됩니다. (채널 수, 높이, 너비) 형태).

이미지는 보통 3D 텐서로 표현됩니다. RGB 이미지는 (높이, 너비, 채널 수)로 나타낼 수 있으며, 예를 들어, 256x256 크기의 RGB 이미지는 (256, 256, 3) 크기의 텐서로 변환됩니다.
배치 데이터는 여러 이미지들을 하나로 묶어 4D 텐서로 표현될 수 있습니다. 예를 들어, 배치 크기가 32인 경우, (32, 256, 256, 3) 형태의 텐서가 됩니다.


배치 단위로 처리한다"는 것은 한 번에 여러 데이터를 묶어서 처리하는 방식을 의미합니다. 딥러닝에서는 데이터를 배치(batch) 단위로 처리하는 것이 일반적입니다. 배치 처리는 주로 훈련 효율성과 메모리 관리를 개선하는 데 중요한 역할을 합니다.

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

Generator는 이미지를 압축한 뒤 복원하는 모델입니다.
Encoder와 Decoder를 사용하여 이미지를 잠재 공간(latent space)으로 압축하고, 다시 복원하여 원래 이미지를 재구성합니다

이미지의 중요한 특성이나 패턴을 학습하여 원본 이미지를 **재구성(reconstruction)**하는 방식으로, 이미지의 특징을 잘 파악하고 불필요한 세부 사항을 제거하는 데 도움을 주기 때문입니다.


**잠재 공간(latent space)**은 데이터를 압축한 후 얻어진 특성 공간을 의미합니다. 즉, 원본 데이터의 중요한 정보만을 담고 있는 저차원 공간입니다. 이 공간에서 각 데이터 포인트는 벡터 형태로 나타납니다.


1. 잠재 공간의 역할:
잠재 공간은 원본 데이터를 압축한 후의 정보가 저장되는 공간입니다. 예를 들어, 이미지의 잠재 공간 벡터는 해당 이미지의 중요한 특성(예: 색상, 모양, 구조 등)을 압축하여 표현합니다.
이 공간에서의 각 벡터는 해당 이미지가 가지고 있는 특징을 요약한 것이며, 이 벡터를 **디코딩(decoding)**하여 원래의 이미지를 복원할 수 있습니다.
2. 잠재 공간의 예시:
Autoencoder에서는 인코더가 이미지를 잠재 공간으로 압축하고, 디코더가 이를 복원합니다. 잠재 공간에서의 벡터는 이미지의 핵심적인 정보만을 담고 있으며, 고차원 이미지 데이터를 저차원으로 압축하여 더 쉽게 학습할 수 있도록 합니다.
GAN에서는 **생성자(generator)**가 잠재 공간에서 샘플을 생성하여 이미지를 만들어냅니다. 잠재 공간은 생성자가 새로운 이미지를 생성할 수 있는 공간으로 활용됩니다.
3. 잠재 공간에서의 벡터:
예를 들어, 고양이 이미지를 압축하여 잠재 공간 벡터로 변환했다면, 이 벡터는 고양이의 색, 모양, 크기 등의 중요한 특성을 요약한 값들을 가질 것입니다. 그리고 이 벡터는 디코더에 의해 다시 고양이 이미지를 복원할 수 있게 됩니다.

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),
            nn.Flatten(),
            nn.Linear(256 * (image_size // 8) * (image_size // 8), 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

Discriminator는 이미지가 실제 데이터인지, 생성된 데이터인지를 구별하는 모델입니다.
LeakyReLU 활성화 함수와 Sigmoid를 사용하여 최종 판별 결과를 출력합니다.

In [ ]:
def train_ganomaly(generator, discriminator, train_loader, num_epochs, lr, device):
    generator = generator.to(device)
    discriminator = discriminator.to(device)

    g_optimizer = torch.optim.Adam(generator.parameters(), lr=lr)
    d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=lr)

    criterion = nn.MSELoss()

    for epoch in range(num_epochs):
        g_loss_epoch = 0.0
        d_loss_epoch = 0.0

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            images, _ = batch
            images = images.to(device)

            # 1. Train Discriminator
            d_optimizer.zero_grad()

            real_labels = torch.ones((images.size(0), 1), device=device)
            fake_labels = torch.zeros((images.size(0), 1), device=device)

            real_outputs = discriminator(images)
            real_loss = criterion(real_outputs, real_labels)

            fake_images = generator(images)
            fake_outputs = discriminator(fake_images.detach())
            fake_loss = criterion(fake_outputs, fake_labels)

            d_loss = real_loss + fake_loss
            d_loss.backward()
            d_optimizer.step()

            # 2. Train Generator
            g_optimizer.zero_grad()

            fake_outputs = discriminator(fake_images)
            reconstruction_loss = criterion(fake_images, images)
            g_loss = reconstruction_loss + criterion(fake_outputs, real_labels)
            g_loss.backward()
            g_optimizer.step()

            g_loss_epoch += g_loss.item()
            d_loss_epoch += d_loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], G Loss: {g_loss_epoch:.4f}, D Loss: {d_loss_epoch:.4f}")

    return generator

GANomaly 학습 함수입니다.
Discriminator와 Generator를 번갈아 학습시킵니다. Discriminator는 진짜와 가짜를 구별하고, Generator는 가짜 이미지를 생성하여 Discriminator를 속이려고 합니다.
각 모델에 대해 최적화 과정과 손실 함수(MSE Loss)를 계산하고, Generator의 성능을 점차적으로 향상시킵니다.

전체 학습은 num_epochs번 반복되며, 각 에포크마다 데이터를 배치 단위로 모델에 전달하면서 학습이 이루어집니다.
매 배치마다 Generator와 Discriminator를 학습시키고, 그에 대한 손실 값을 계산합니다.
각 배치가 끝날 때마다 Generator와 Discriminator의 손실을 누적하여, 에포크가 끝날 때마다 그 값을 출력하여 학습 상황을 추적합니다.

에포크는 학습 데이터를 한 번 전체 사용하는 반복 단위입니다.
배치는 데이터를 작은 덩어리로 나누어 한 번에 모델에 전달하여 학습하는 단위입니다.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = Generator()
discriminator = Discriminator()

trained_generator = train_ganomaly(generator, discriminator, train_loader, num_epochs=50, lr=0.0002, device=device)

Epoch 1/50:  57%|█████▋    | 4/7 [02:40<01:55, 38.56s/it]

모델을 학습합니다. cuda가 가능하면 GPU를 사용하고, 그렇지 않으면 CPU로 학습을 진행합니다.

In [ ]:
def detect_anomalies(generator, test_loader, threshold, device):
    generator = generator.to(device)
    generator.eval()

    anomaly_scores = []
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Detecting anomalies"):
            images, _ = batch
            images = images.to(device)

            reconstructed_images = generator(images)
            scores = torch.mean((reconstructed_images - images) ** 2, dim=[1, 2, 3])  # MSE Loss
            anomaly_scores.extend(scores.cpu().numpy())

    predictions = (np.array(anomaly_scores) > threshold).astype(int)
    return anomaly_scores, predictions

테스트 데이터셋에서 Anomaly Detection

In [ ]:
threshold = 0.1
anomaly_scores, predictions = detect_anomalies(trained_generator, test_loader, threshold, device)

# 결과 저장
result_df = pd.DataFrame({
    "ID": [f"TEST_{i:03d}.png" for i in range(len(predictions))],
    "label": predictions
})
result_df.to_csv("sample_submission.csv", index=False)
print("Results saved to sample_submission.csv")